In [1]:
import PyPDF2
import requests
import numpy as np, seaborn as sns, pandas as pd
import nltk, nltk.sentiment, sklearn
import collections
%matplotlib inline

In [2]:
#convert list to pd.series
#doc1_str = pd.Series([''.join(doc1)])

#Preprocessing - tokenize and lower case pdf-strings
tokenizer = nltk.tokenize.TweetTokenizer()
def preprocessing(string):
    string = string.lower()
    string = tokenizer.tokenize(string)
    return string

#Apply positive/negative sets on pdf-string
#tok_doc1 = doc1_str.apply(preprocessing)

def count_dictionary(document_elem, dictionary):
    lst = []
    for word in document_elem:
        if (word in dictionary):
            lst.append(word)
    return len(lst)


In [3]:
#PDF extractor script. Creates a list separated per page.
def pdf_extractor(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = collections.Counter(range(number_of_pages))
    res1 = []
    for i in range(len(c)):
        page = read_pdf.getPage(i)
        page_content = page.extractText()
        res1.append(page_content)
    return res1
   

In [4]:
import requests

#Create set for negative and positive lexicon
negative = set(requests.get('http://ptrckprry.com/course/ssd/data/negative-words.txt').text.split(';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;')[2].split('\n')[2:])
positive = set(requests.get('http://ptrckprry.com/course/ssd/data/positive-words.txt').text.split(';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;')[2].split('\n')[2:])

In [9]:
import os

#Sebastian file path
basedir = r'C:\Users\sebag\Documents\GitHub\SDS Group 35\Exam_Project\Minutes_PDF'
testdir = r'C:\Users\sebag\Documents\GitHub\Exam\Test folder'

def sentiment_analysis(pos, neg, file_path):
    pos_words = []
    neg_words = []
    average_tone = []
    i = 0
    error = []
    
    for fn in os.listdir(file_path):
        try:
            print('Now treating file: ' + fn + '\nIterate: ' + str(i))
            doc = pdf_extractor(file_path + '\\' + fn)
            doc = pd.Series([''.join(doc)])
            doc = doc.apply(preprocessing)

            pos_words.append(int(count_dictionary(doc[0], pos)))
            neg_words.append(int(count_dictionary(doc[0], neg)))
            average_tone.append((pos_words[i]-neg_words[i])/len(doc[0]))
            error.append(False)  
            
            i += 1
        
        except KeyError:
            error.append(True)
            pos_words.append(np.NaN)
            neg_words.append(np.NaN)
            average_tone.append(np.NaN)
            
            i += 1
            
        #date = re.findall(r'\d+', fn)
    df = pd.DataFrame(zip(error, pos_words, neg_words, average_tone, error))
    return df

sent_df = sentiment_analysis(positive, negative, basedir)
sent_df


Now treating file: ppp000203e.pdf
Iterate: 0
Now treating file: ppp000322e.pdf
Iterate: 1
Now treating file: ppp000504e.pdf
Iterate: 2
Now treating file: ppp000607e.pdf
Iterate: 3
Now treating file: ppp000706e.pdf
Iterate: 4
Now treating file: ppp000816e.pdf
Iterate: 5
Now treating file: ppp001024e.pdf
Iterate: 6
Now treating file: ppp001220e.pdf
Iterate: 7
Now treating file: ppp010201e.pdf
Iterate: 8
Now treating file: ppp010326e.pdf
Iterate: 9
Now treating file: ppp010426e.pdf
Iterate: 10
Now treating file: ppp010530e.pdf
Iterate: 11
Now treating file: ppp010614e.pdf
Iterate: 12
Now treating file: ppp010705e.pdf
Iterate: 13
Now treating file: ppp010823e.pdf
Iterate: 14
Now treating file: ppp010917e.pdf
Iterate: 15
Now treating file: ppp011015e.pdf
Iterate: 16
Now treating file: ppp011108e.pdf
Iterate: 17
Now treating file: ppp011204e.pdf
Iterate: 18
Now treating file: ppp020207e.pdf
Iterate: 19
Now treating file: ppp020318e.pdf
Iterate: 20
Now treating file: ppp020425e.pdf
Iterate: 2

,0,1,2,3
0,False,122.0,85.0,0.006086
1,False,70.0,41.0,0.006694
2,False,70.0,48.0,0.004838
3,False,58.0,51.0,0.001999
4,False,76.0,64.0,0.002416
5,False,58.0,51.0,0.001999
6,False,58.0,61.0,-0.000741
7,False,113.0,143.0,-0.005166
8,False,85.0,118.0,-0.005850
9,False,55.0,110.0,-0.013067


In [13]:
sent_df.columns = ['Number of positive words', 'Number of negative words', 'Average tone ratio', 'Error in processing PDF']
print(sent_df.loc[sent_df['Number of positive words'] == True])
#sent_df

     Number of positive words  Number of negative words  Average tone ratio  \
94                       True                       NaN                 NaN   
95                       True                       NaN                 NaN   
96                       True                       NaN                 NaN   
97                       True                       NaN                 NaN   
98                       True                       NaN                 NaN   
100                      True                       NaN                 NaN   
101                      True                       NaN                 NaN   
102                      True                       NaN                 NaN   
103                      True                       NaN                 NaN   
104                      True                       NaN                 NaN   
105                      True                       NaN                 NaN   
106                      True                       

In [29]:
pos_words = []
doc = pdf_extractor(r'C:\Users\sebag\Documents\GitHub\SDS Group 35\Exam_Project\Minutes_PDF\ppp000203e.pdf')
doc = pd.Series([''.join(doc)])
doc = doc.apply(preprocessing)
count_dictionary(doc[0], negative)
pos_words.append(count_dictionary(doc[0], positive))
print(pos_words)
print(len(doc[0]))

[122]
6080


In [38]:

pos_words = []
neg_words = []
average_tone = []
i = 0

for fn in os.listdir(testdir):
    doc = pdf_extractor(testdir + '\\' + fn)
    doc = pd.Series([''.join(doc)])
    doc = doc.apply(preprocessing)
    pos_words.append(count_dictionary(doc[0], positive))
    neg_words.append(count_dictionary(doc[0], negative))
    average_tone = (pos_words[i]-neg_words[i])/len(doc[0])
        
    i += 1
    #date = re.findall(r'\d+', fn)
df = pd.DataFrame(zip(pos_words, neg_words, average_tone))
df

TypeError: zip argument #3 must support iteration

In [14]:
import os, re, pandas as pd, ast, datetime

#Mark file path:
path = 'C:/Users/maje.DKSUND/Desktop/SDS-Group-35-master_new/SDS-Group-35-master/Exam_Project/Minutes_PDF'

files = []
files2 = []

# r=root, d=directories, f = files
for r, d, f in os.walk(basedir):
    for file in f:
        if '.pdf' in file:
            files.append(file[7:9] + "-" + file[5:7] + "-" + file[3:5])
            files2.append(file)

Final_list = {'Dato':files,'Filnavn':files2}

df_files = pd.DataFrame(Final_list)
df_files['date_time'] = pd.to_datetime(df_files['Dato'])
df_files['date_time_announcement'] = pd.to_datetime(df_files['Dato']) + pd.DateOffset(days=1)
df_files

,Dato,Filnavn,date_time,date_time_announcement
0,03-02-00,ppp000203e.pdf,2000-03-02,2000-03-03
1,22-03-00,ppp000322e.pdf,2000-03-22,2000-03-23
2,04-05-00,ppp000504e.pdf,2000-04-05,2000-04-06
3,07-06-00,ppp000607e.pdf,2000-07-06,2000-07-07
4,06-07-00,ppp000706e.pdf,2000-06-07,2000-06-08
5,16-08-00,ppp000816e.pdf,2000-08-16,2000-08-17
6,24-10-00,ppp001024e.pdf,2000-10-24,2000-10-25
7,20-12-00,ppp001220e.pdf,2000-12-20,2000-12-21
8,01-02-01,ppp010201e.pdf,2001-01-02,2001-01-03
9,26-03-01,ppp010326e.pdf,2001-03-26,2001-03-27
